<a href="https://colab.research.google.com/github/kra3810/The-Puppet-Master/blob/main/wav2lip_colab_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/camenduru/Wav2Lip
!pip install -q gradio==3.50.2 yt_dlp ffmpeg-python librosa==0.8.0
%cd Wav2Lip

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 69 (from 1)
Unpacking objects: 100% (72/72), 442.42 KiB | 1.86 MiB/s, done.
Filtering content: 100% (6/6), 936.88 MiB | 88.85 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import gradio as gr
from yt_dlp import YoutubeDL
import os

def download_video(url):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':'/content/video.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(url)
    return f"/content/video.mp4"

def generate(audio_in):
    print(audio_in)
    os.system(f"python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/video.mp4' --audio '{audio_in}'")
    return f"/content/Wav2Lip/results/result_voice.mp4"

with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      input_text = gr.Textbox(show_label=False, value="https://youtu.be/EU3hIXXeiz4")
      input_download_button = gr.Button(value="Download from YouTube or Twitch")
      audio_in = gr.Audio(show_label=False, type='filepath')
      input_generate_button = gr.Button(value="Generate")
    with gr.Column():
        video_out = gr.Video(label="Output Video")
    input_download_button.click(download_video, inputs=[input_text], outputs=[video_out])
    input_generate_button.click(generate, inputs=[audio_in], outputs=[video_out])

demo.queue().launch(inline=False, share=True, debug=True)

In [ ]:
import os
!sed -i '1i import numpy as np; np.float = float; np.int = int; np.complex = complex' /content/Wav2Lip/audio.py
!sed -i '1i import numpy as np; np.float = float; np.int = int; np.complex = complex' /content/Wav2Lip/inference.py
!sed -i '1i import numpy as np; np.float = float; np.int = int; np.complex = complex' /usr/local/lib/python3.12/dist-packages/librosa/util/utils.py

In [ ]:
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/video.mp4" --audio "/content/audio.mp3" --resize_factor 1 --outfile "result.mp4"

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 634
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-lib

In [ ]:
import torch
print("GPU ใช้งานได้ไหม:", torch.cuda.is_available())
# ถ้าขึ้น True แปลว่าพร้อมลุย
# ถ้าขึ้น False แปลว่า Colab ยังไม่จ่าย GPU ให้ (ต้องไปตั้งค่า Runtime ใหม่)

GPU ใช้งานได้ไหม: True


In [ ]:
import os
import shutil
import subprocess

# ตั้งค่า Path
input_video = "/content/Wav2Lip/result.mp4"
output_video = "/content/Wav2Lip/result_hd_final.mp4"
temp_input_frames = "/content/temp_frames_in"
temp_output_frames = "/content/temp_frames_out"

# 1. เคลียร์พื้นที่ (ลบของเก่าทิ้งถ้ามี)
if os.path.exists(temp_input_frames): shutil.rmtree(temp_input_frames)
if os.path.exists(temp_output_frames): shutil.rmtree(temp_output_frames)
os.makedirs(temp_input_frames)
os.makedirs(temp_output_frames)

print("🎬 ขั้นตอนที่ 1: แยกวิดีโอเป็นรูปภาพ (Extract Frames)...")
# ใช้ FFmpeg ระเบิดไฟล์
cmd_extract = f'ffmpeg -i "{input_video}" -qscale:v 1 -qmin 1 -qmax 1 -vsync 0 "{temp_input_frames}/frame_%04d.png" -loglevel error'
subprocess.call(cmd_extract, shell=True)

print(f"✅ แยกเสร็จแล้ว! มีทั้งหมด {len(os.listdir(temp_input_frames))} รูป")

print("\n✨ ขั้นตอนที่ 2: ส่งรูปไปชุบตัวให้ชัด (Upscaling with GFPGAN)...")
# ย้ายไปโฟลเดอร์โปรแกรม
%cd /content/Wav2Lip/GFPGAN
# สั่งรัน GFPGAN ใส่โฟลเดอร์รูปภาพ (ไม่ใช่ไฟล์วิดีโอแล้ว)
cmd_upscale = f'python inference_gfpgan.py -i "{temp_input_frames}" -o "{temp_output_frames}" -v 1.3 -s 2'
subprocess.call(cmd_upscale, shell=True)

print("\n🎞️ ขั้นตอนที่ 3: รวมร่างกลับเป็นวิดีโอ (Merge back to Video)...")
# path ที่รูปชัดๆ ถูกเก็บไว้
restored_frames_path = f"{temp_output_frames}/restored_imgs/frame_%04d.png"

# ใช้ FFmpeg รวมร่าง + เอาเสียงจากคลิปต้นฉบับมาใส่
cmd_merge = f'ffmpeg -y -r 25 -i "{restored_frames_path}" -i "{input_video}" -map 0:v -map 1:a -c:v libx264 -pix_fmt yuv420p -c:a copy "{output_video}" -loglevel error'
subprocess.call(cmd_merge, shell=True)

# เช็คผลลัพธ์
if os.path.exists(output_video):
    print("\n🎉🎉🎉 สำเร็จแล้วววว!! (ของจริงแบบ Final) 🎉🎉🎉")
    print(f"ไฟล์ HD อยู่ที่: {output_video}")
    print("👉 ไปที่โฟลเดอร์ Wav2Lip (แถบซ้าย) กดรีเฟรช แล้วโหลดไฟล์ result_hd_final.mp4 ได้เลย!")
else:
    print("\n❌ ยัง Error... แปลกมาก แคปหน้าจอมาดูหน่อยครับ")

🎬 ขั้นตอนที่ 1: แยกวิดีโอเป็นรูปภาพ (Extract Frames)...
✅ แยกเสร็จแล้ว! มีทั้งหมด 490 รูป

✨ ขั้นตอนที่ 2: ส่งรูปไปชุบตัวให้ชัด (Upscaling with GFPGAN)...
/content/Wav2Lip/GFPGAN

🎞️ ขั้นตอนที่ 3: รวมร่างกลับเป็นวิดีโอ (Merge back to Video)...

🎉🎉🎉 สำเร็จแล้วววว!! (ของจริงแบบ Final) 🎉🎉🎉
ไฟล์ HD อยู่ที่: /content/Wav2Lip/result_hd_final.mp4
👉 ไปที่โฟลเดอร์ Wav2Lip (แถบซ้าย) กดรีเฟรช แล้วโหลดไฟล์ result_hd_final.mp4 ได้เลย!
